In [2]:
# ============================================================
# 🧠 Step 1 — Imports
# ============================================================
import pandas as pd
from sklearn.model_selection import train_test_split
import os
os.environ["WANDB_MODE"] = "disabled"  # disables WandB logging

# ============================================================
# 📂 Step 2 — Load the raw Capterra dataset
# ============================================================
df = pd.read_csv('/content/capterra_reviews.csv')

# Keep only the relevant columns
df = df[['overall_text', 'overall_rating']].dropna()

# Create binary label: 1 if rating >=4 else 0
df['label'] = (df['overall_rating'] >= 4).astype(int)
df = df[['overall_text', 'label']].rename(columns={'overall_text': 'text'})

# ============================================================
# ✂️ Step 3 — Train/Validation/Test Split
# ============================================================
train, temp = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
val, test = train_test_split(temp, test_size=0.5, stratify=temp['label'], random_state=42)

train.to_csv('/content/train.csv', index=False)
val.to_csv('/content/val.csv', index=False)
test.to_csv('/content/test.csv', index=False)

print("✅ Train/Val/Test CSVs recreated successfully!")


✅ Train/Val/Test CSVs recreated successfully!


In [5]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer
)
from datasets import load_dataset
import evaluate
import numpy as np

# ============================================================
# Load the newly created CSVs
# ============================================================
data_files = {
    "train": "/content/train.csv",
    "validation": "/content/val.csv",
    "test": "/content/test.csv"
}
datasets = load_dataset("csv", data_files=data_files)

# ============================================================
# Tokenization
# ============================================================
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = datasets.map(preprocess, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ============================================================
# Model
# ============================================================
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=2)

# ============================================================
# Metrics
# ============================================================
accuracy = evaluate.load("accuracy")
roc_auc = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = np.exp(logits) / np.exp(logits).sum(-1, keepdims=True)
    preds = np.argmax(logits, axis=1)
    auc = roc_auc.compute(prediction_scores=probs[:, 1], references=labels)["roc_auc"]
    acc = accuracy.compute(predictions=preds, references=labels)["accuracy"]
    return {"accuracy": acc, "roc_auc": auc}

# ============================================================
# TrainingArguments (local only)
# ============================================================
training_args = TrainingArguments(
    output_dir="./bert-review-sentiment",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=50,
    save_steps=500,
    report_to="none"  # disable WandB & TensorBoard
)

# ============================================================
# Trainer
# ============================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# ============================================================
# Train + Evaluate
# ============================================================
trainer.train()

val_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["validation"])
print("Validation:", val_metrics)

test_metrics = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
print("Test:", test_metrics)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/tmp/ipython-input-4155543890.py:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.309800
100,0.258000
150,0.282700
200,0.216500
250,0.263100
300,0.214800
350,0.321700
400,0.221000
450,0.228800
500,0.238400


Validation: {'eval_loss': 0.22835670411586761, 'eval_accuracy': 0.9430379746835443, 'eval_roc_auc': 0.5476323639075318, 'eval_runtime': 0.7574, 'eval_samples_per_second': 417.201, 'eval_steps_per_second': 52.81, 'epoch': 5.0}
Test: {'eval_loss': 0.2386857271194458, 'eval_accuracy': 0.9400630914826499, 'eval_roc_auc': 0.5802719886965737, 'eval_runtime': 0.7441, 'eval_samples_per_second': 426.0, 'eval_steps_per_second': 53.754, 'epoch': 5.0}


In [4]:
%pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
